In [ ]:
import numpy as np
import pandas as pd
import altair as alt
from math import exp, sin

In [ ]:
# parametr for increasing altair charts capacity
alt.data_transformers.enable('default', max_rows=10000000)

In [ ]:
def delta_phi(phi, param):
    delta_phi = np.exp(-np.sin(phi) / param)
    return delta_phi

---

In [ ]:
def simulation(param=1, steps=200, init_phi=0):
    """Simulation of process with specific params"""
    n = np.zeros(steps + 1)
    phi = np.zeros(steps + 1)
    n[0] = 0
    phi[0] = init_phi
    
    for i in range(steps):
        phi[i+1] = (phi[i] + delta_phi(phi[i], param)) % (2 * np.pi)
        n[i+1] = n[i] + 1
    
    df = pd.DataFrame({'n': n, 'phi': phi})

    y_ticks = [0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi]
    chart = alt.Chart(df).mark_line().encode(
        x=alt.X('n:Q', title='iteration number'),
        y=alt.Y('phi:Q', title='phi',
            axis=alt.Axis(values=y_ticks, labels=True, tickCount=5, labelExpr="datum.value == 0 ? '0' : datum.value == 3.141592653589793 ? 'π' : datum.value == 6.283185307179586 ? '2π' : datum.value == 1.5707963267948966 ? 'π/2' : datum.value == 4.71238898038469 ? '3π/2' : datum.value"))
        ).properties(
        title=f'Param = {param}').interactive()
    chart.save(f'data_param_{param}_init_{init_phi}.png', scale_factor=2)
    chart.display()

In [ ]:
simulation(2, 1000)

In [ ]:
steps = 500
for param in [0.1, 0.5, 0.7, 0.9, 1, 1.2, 1.4, 1.5, 1.6, 1.9, 2.0, 2.3, 2.5, 3.0, 4.0, 5.5, 6.0, 9.0]:
    simulation(param, steps)

In [ ]:
def create_logistic_map(param_min, param_max, steps=2000, save_last=800):
    """Create logistic map by Phi and save in PNG and SVG"""
    params = np.linspace(param_min, param_max, 400)
    map_data = []
    phi_init = 0

    for param in params:
        phi = phi_init
        phi_values = []
        for i in range(steps):
            phi = (phi + delta_phi(phi, param)) % (2 * np.pi)
            if i >= (steps - save_last):
                phi_values.append(phi)
        
        for phi_value in phi_values:
            map_data.append({'param': param, 'phi': phi_value})

    
    df_logistic_map = pd.DataFrame(map_data)
    y_ticks = [0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi]
    chart = alt.Chart(df_logistic_map).mark_point(size=0.1, opacity=0.5).encode(
        x=alt.X('param:Q', title='param', scale=alt.Scale(domain=[param_min, param_max])),
        y=alt.Y('phi:Q', title='phi',
                axis=alt.Axis(values=y_ticks, labels=True, tickCount=5, labelExpr="datum.value == 0 ? '0' : datum.value == 3.141592653589793 ? 'π' : datum.value == 6.283185307179586 ? '2π' : datum.value == 1.5707963267948966 ? 'π/2' : datum.value == 4.71238898038469 ? '3π/2' : datum.value"))
    ).properties(
        title=f'{param_min}-{param_max} Logistic map: phi =  (phi + exp(-sin(phi) / param)) % (2 * pi), last {save_last}',
        width=800,
        height=600
    )#.interactive()
    #chart.save(f'logistic_map_{phi_init}_{param_min}-{param_max}.json')
    chart.save(f'logistic_map_{param_min}-{param_max}_last_{save_last}.png', scale_factor=2)
    chart.save(f'logistic_map_{param_min}-{param_max}.svg')
    chart.display()

In [ ]:
create_logistic_map(31, 33)

In [ ]:
create_logistic_map(30, 100)

---
### Tests with matplotlib

In [ ]:
from matplotlib import pyplot as plt
from math import exp, sin
import numpy as np

In [ ]:
steps = 100
param = 0.5

n = np.zeros(steps + 1)
phi = np.zeros(steps + 1)
n[0], phi[0] = 0, 0.5


for i in range(steps):
    phi[i+1] = (phi[i] + delta_phi(phi[i], param)) % (2 * np.pi)
    n[i+1] = n[i] + 1
    
fig, ax = plt.subplots()
ax.plot(n, phi, alpha=0.5)
ax.set(xlabel='Period (n)', ylabel='Phi')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

param_min = 0.1
param_max = 1.1
param_values = np.linspace(param_min, param_max, 10000)

iterations = 3000
last = 1000

phi = 0
param_list = []
phi_list = []

for i in range(iterations):
    phi = (phi + np.sign(np.sin(phi)) * np.exp(-np.sin(phi))/param_values) % (2 * np.pi)
    # phi = (phi + np.sign(np.sin(phi)) * np.exp(np.sin(phi))/param_values) % (2*np.pi)
    # phi = (phi + np.sign(np.sin(phi)) * np.exp( np.abs(np.sin(phi))/param ) ) % (2*np.pi)

    if i >= (iterations - last):
        param_list.extend(param_values)
        phi_list.extend(phi)

plt.figure(figsize=(10, 7))
plt.plot(param_list, phi_list, ',k', alpha=0.5)
plt.title("Logistic map: phi =  (phi + exp(-sin(phi) / param)) % (2 * pi)")
plt.xlabel("param")
plt.ylabel("phi")
plt.yticks(np.linspace(0, 2 * np.pi, 5), ['0', 'π/2', 'π', '3π/2', '2π'])
plt.show()